In [1]:
####  merge ip and input raw read counts into a matrix
os.chdir("/Charles/project/ASm6A/by_ind/count/")
input_dir = "/Charles/project/ASm6A/by_ind/count/input_count/rename/"
ip_dir = "/Charles/project/ASm6A/by_ind/count/ip_count/rename/"
###
result_dir = "read_matrix/"
os.system("mkdir -p %s" % result_dir)


# filter based on read counts > 5;
def preprocess_count(in_file, file_type):  # file_type == "ip" or "input"
    df = pd.read_table(in_file)
    df["ID"] = df["contig"] + ":" + df["position"].astype(str)
    df_sub = df[["ID", "refCount", "altCount"]]
    df_sub = df_sub[(df_sub['refCount'] > 5) &
                    (df_sub['altCount'] > 5)]  ############
    df_sub.columns = [
        "ID", "refCount_%s" % file_type,
        "altCount_%s" % file_type
    ]
    return df_sub


ip_list = glob.glob("%s/*.readcounts.txt" % ip_dir)
for ip_count in ip_list:
    ip_prefix = os.path.basename(ip_count).split(".")[0]
    result_file = os.path.join(result_dir, "%s.txt" % ip_prefix)
    input_count = os.path.join(input_dir, os.path.basename(ip_count))
    df_input = preprocess_count(input_count, "input")
    df_ip = preprocess_count(ip_count, "ip")
    df = df_input.merge(df_ip, on="ID")
    #     ip_totalNum, input_totalNum = int(totalNum_dict[ip_prefix][0]), int(totalNum_dict[ip_prefix][2])
    df.to_csv(result_file, sep="\t", index=False)

In [4]:
#### generate matrix
os.chdir("/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/count/")
txt_list = glob.glob("*.readcounts.txt")
result_dir = "matrix"
os.system("mkdir -p %s" % result_dir)

def format_file(in_file):
    prefix = os.path.basename(in_file).split(".readcounts.txt")[0]
    df = pd.read_table(in_file)
    df = df[['contig','position','refAllele','altAllele','refCount','altCount']]
    df["ref_%s"%prefix] = df['refCount']
    df["alt_%s"%prefix] = df['altCount']
    del df['refCount']
    del df['altCount']
    return df

df = format_file(txt_list[0])
for txt in txt_list[1:]:
    df_i = format_file(txt)
    df = df.merge(df_i, on=['contig','position','refAllele','altAllele'], how="outer")
res = os.path.join(result_dir, "totalSamples.txt")
df.to_csv(res, sep="\t", index=False)